In [3]:
def hamming(seq1, seq2):
    return sum([1 for (c1, c2) in zip(seq1, seq2) if c1 != c2])

In [4]:
genomes = ['ACGAGCTACGTAC', 'CCGTACGATGCTA', 'GCACAGATTCGGACT' , 'TTGCGGACCGTACGTAGC']

In [5]:
k = 6

In [6]:
"ACGGTC"

'ACGGTC'

# Motif finding problem

Istance: a set $S$ of $n$ genomes (each represented by a string), an integer $k\ge 1$

Feasible solution: a substring $T_i$ for each string $S_i\in S$

Goal: is to minimize $\sum_i d_H(T_i, m)$, where $m$ is the consensus of the subtrings $T_i$. The string $m$ is also the motif found.

In [7]:
samples = ['ACGTAC', 'ATGCTA', 'CGGACT' , 'CGTAGC']

'ACGTAC' 2
'ATGCTA' 4
'CGGACT' 3
'CGTAGC' 3
---------
'AGGAAC'
 223212 highest frequency
 221232 <- sum those values to obtain the sum of the hamming distances

In [8]:
from collections import Counter

def consensus(strings):
    consensus = ''
    for pos in range(len(strings[0])):
        count = Counter([seq[pos] for seq in strings])
        consensus += count.most_common(1)[0][0]
    return consensus 

In [9]:
consensus(samples)

'AGGAAC'

In [10]:
def total_distance(cons, strings):
    return sum([hamming(cons, seq) for seq in strings])

In [11]:
total_distance("CCCCCC", samples)

17

In [12]:
def consensus_distance(strings):
    cons = consensus(strings)
    return sum([hamming(cons, seq) for seq in strings])

In [43]:
def consensus_distance2(strings):
    distance = 0
    for pos in range(len(strings[0])):
        count = Counter([seq[pos] for seq in strings])
        distance += (len(strings) - max(count.values()))
    return distance 

In [16]:
consensus_distance2(samples)

12

In [17]:
x = Counter("ACGCTAGTATTACGATGGCGATCATTTACGAC")
x

Counter({'A': 9, 'C': 7, 'G': 7, 'T': 9})

In [18]:
x.values()

dict_values([9, 7, 7, 9])

In [19]:
max(x.values())

9

Modify `consensus_distance2` to compute the number of characters of the input strings that are **equal** to the corresponding character of the consensus.

In [20]:
def similarity(strings):
    value = 0
    for pos in range(len(strings[0])):
        count = Counter([seq[pos] for seq in strings])
        value += max(count.values())
    return value

Trivial approach: compute all susbstrings of `genomes`

In [21]:
genomes[0]

'ACGAGCTACGTAC'

In [22]:
for genome in genomes:
    print([ genome[i: i + k] for i in range(len(genome) - k ) ])

['ACGAGC', 'CGAGCT', 'GAGCTA', 'AGCTAC', 'GCTACG', 'CTACGT', 'TACGTA']
['CCGTAC', 'CGTACG', 'GTACGA', 'TACGAT', 'ACGATG', 'CGATGC', 'GATGCT']
['GCACAG', 'CACAGA', 'ACAGAT', 'CAGATT', 'AGATTC', 'GATTCG', 'ATTCGG', 'TTCGGA', 'TCGGAC']
['TTGCGG', 'TGCGGA', 'GCGGAC', 'CGGACC', 'GGACCG', 'GACCGT', 'ACCGTA', 'CCGTAC', 'CGTACG', 'GTACGT', 'TACGTA', 'ACGTAG']


In [28]:
positions = [0] * len(genomes)
level = len(genomes) - 1
substrings = [genomes[i][pos: pos + k] for (i, pos) in enumerate(positions)]
best_value = consensus_distance2(substrings)
best = consensus(substrings)

while(level >= 0):
    substrings = [genomes[i][pos: pos + k] for (i, pos) in enumerate(positions)]
    distance = consensus_distance2(substrings)
    if distance < best_value:
        best = consensus(substrings)
        best_value = distance
        print(substrings, best, distance, positions)

        
    # positions[0] -> substring[0]
    # positions[1] -> substring[1]   ________ the first two substrings are not too different
    # positions[2] -> substring[2]   ________ the first three substrings already are too different <--- change this substring
    # positions[3] -> substring[3]
    level = len(genomes) - 1
    while level >= 0 and consensus_distance2(substrings[:level + 1]) >= best_value:
        level -= 1
    level += 1
    for i in range(level + 1, len(genomes)):
        positions[i] = 0
        
    while level >= 0 and positions[level] >= len(genomes[level]) - k:
        positions[level] = 0
        level -= 1
    
    if level >= 0:
        positions[level] += 1

print(best, best_value)

['ACGAGC', 'CCGTAC', 'GCACAG', 'GCGGAC'] GCGAAC 8 [0, 0, 0, 2]
['ACGAGC', 'CCGTAC', 'GCACAG', 'CCGTAC'] CCGTAC 7 [0, 0, 0, 7]
['ACGAGC', 'CCGTAC', 'TCGGAC', 'GCGGAC'] ACGGAC 6 [0, 0, 8, 2]
['ACGAGC', 'CCGTAC', 'TCGGAC', 'CCGTAC'] CCGTAC 5 [0, 0, 8, 7]
['GCTACG', 'CGTACG', 'CGGACT', 'CGTACG'] CGTACG 4 [4, 1, 9, 8]
['ACGTAC', 'CCGTAC', 'TCGGAC', 'CCGTAC'] CCGTAC 3 [7, 0, 8, 7]
CCGTAC 3


In [80]:
def best_substring(long, short):
    l = len(short)
    # find the l-long substring s of long that minimizes dh(s, short)
    best = long[:l] 
    best_value = hamming(best, short)
    for i in range(len(long) - l + 1):
        candidate = long[i: i + l]
        if hamming(short, candidate) < best_value:
            best = candidate
            best_value = hamming(best, short)
    return best

# Exercise

Modify the approach to iterate over all k-long candidate consensus strings

In [116]:
candidate_consensus = ['A'] * k
pos = k - 1
next_char = {'A': 'C', 'C': 'G', 'G': 'T' }

substrings = [best_substring(genome, ''.join(candidate_consensus[:pos + 1])) for genome in genomes]
best_value = total_distance(''.join(candidate_consensus[:pos + 1]), substrings)
best = candidate_consensus

while(pos >= 0):
       
    pos = k - 1
    # extract the (pos + 1)-long substring minimized the hamming distance from each genome 
    substrings = [best_substring(genome, ''.join(candidate_consensus[:pos + 1])) for genome in genomes]
    while pos >= 0 and total_distance(''.join(candidate_consensus[:pos + 1]), substrings)  >= best_value:
        pos -= 1
        substrings = [best_substring(genome, ''.join(candidate_consensus[:pos + 1])) for genome in genomes]
    pos += 1  
    # now pos is the index of the character that I want to change
    # 'A' -> 'C'
    # 'C' -> 'G'
    # 'G' -> 'T'
    # 'T' -> special case
    
    
    for i in range(pos + 1, k):
        candidate_consensus[i] = 'A'
        
    while pos >= 0 and candidate_consensus[pos] == 'T':
        candidate_consensus[pos] = 'A'
        pos -= 1
    
    if pos >= 0:
        candidate_consensus[pos] = next_char[candidate_consensus[pos]]

    substrings = [best_substring(genome, ''.join(candidate_consensus)) for genome in genomes]
    distance = total_distance(''.join(candidate_consensus), substrings)
    if distance < best_value:
        best = candidate_consensus.copy()
        best_value = distance
        print(substrings, best, distance)
print(best, best_value)

['ACGAGC', 'CCGTAC', 'ACAGAT', 'GCGGAC'] ['A', 'A', 'A', 'A', 'A', 'C'] 14
['ACGAGC', 'ACGATG', 'GCACAG', 'ACGTAG'] ['A', 'A', 'A', 'A', 'A', 'G'] 13
['GCTACG', 'CGTACG', 'GATTCG', 'GGACCG'] ['A', 'A', 'A', 'A', 'C', 'G'] 12
['ACGAGC', 'CGATGC', 'CACAGA', 'CGTAGC'] ['A', 'A', 'A', 'A', 'G', 'C'] 11
['CTACGT', 'GTACGA', 'ACAGAT', 'GACCGT'] ['A', 'A', 'A', 'C', 'G', 'T'] 10
['TACGTA', 'ACGATG', 'CACAGA', 'ACCGTA'] ['A', 'A', 'C', 'A', 'T', 'A'] 9
['TACGTA', 'TACGAT', 'CACAGA', 'ACCGTA'] ['A', 'A', 'C', 'G', 'T', 'A'] 8
['ACGTAC', 'CCGTAC', 'ACAGAT', 'CCGTAC'] ['A', 'C', 'A', 'T', 'A', 'C'] 7
['ACGTAC', 'CCGTAC', 'TCGGAC', 'GCGGAC'] ['A', 'C', 'G', 'G', 'A', 'C'] 5
['ACGTAC', 'CCGTAC', 'TCGGAC', 'CCGTAC'] ['A', 'C', 'G', 'T', 'A', 'C'] 4
['ACGTAC', 'CCGTAC', 'TCGGAC', 'CCGTAC'] ['C', 'C', 'G', 'T', 'A', 'C'] 3
['C', 'C', 'G', 'T', 'A', 'C'] 3
